# Ciminion2 DRL Gröbner Basis
DRL Gröbner basis computation for an arbitrary Ciminion instance.

In [1]:
load("Ciminion_2.sage")
load("Ciminion_2_polynomial_model.sage")

## Ciminion Instance

In [2]:
ciminion_2 = Ciminion_2(info_level=1)

Ciminion_2 parameters
Field: Finite Field of size 170141183460469231731687303715884105773
Rounds_C: 90
Rounds_E: 14
Constants_C: [[23397008379541153391432454218118545765, 90625764940714311155831092156466871745, 22194738710243333934771351983258090786, 150125624477408065561566494351089945072], [89551371040474592762090349276024179330, 114971290345311656297271588259402762208, 87449813147165763038014530651496322083, 95213023800531571109519063227363262299], [38952127620874089848851863158094952810, 145056436518924332254247252074307871010, 39399078450915408552804726672634832925, 113757883387148997578409111164568762612], [92594428516470812862418648949501073623, 35470143326262006983960190789696000067, 127822126588785297048201343725612854344, 5580044504476043979608161927990605061], [152181577307019295486773126318611591754, 3516570356339553849817949560027357701, 10696860140963233183189861474984234015, 36904873913325295758095183931393930168], [51710543627053454010388801681505327337, 649478077654502

## Ciminion Polynomial model

In [3]:
polys = generate_Ciminion_2_polynomials(ciminion_2=ciminion_2)

P = polys[0].parent()

Plaintext: [148476040925408684745299983510914641396, 27400385091186912883944823885695891979]
Key: [19455829761869325304976509904714170548, 48765002689412583857985067503701580256]
Nonce: 36273157973316374617534923282068955066
Ciphertext: [72130813499958811304693209534699328661, 28062148470498718617964217357908834928]
Term order: degrevlex


## DRL Gröbner Basis computation
First we change to a different DRL term order.

In [4]:
f = efficient_Ciminion_2_termorder(ciminion_2, P)

Next we invert the matrices in the round functions, extract the linear polynomials in each round, and perform Gaussian elimination on the linear polynomials.

In [5]:
polys = [f(poly) for poly in transform_Ciminion_2_polynomial_system(ciminion_2, polys)]

lin_polys = Sequence([poly for poly in polys if poly.degree() == 1])
M, v = lin_polys.coefficients_monomials()

M = M.echelon_form()

lin_polys = ideal((M * v).list())

Now we reduce the non-linear polynomials with respect to the linear polynomials, i.e. we eliminate variables in the non-linear polynomials.
We call the resulting polynomial system the downsized Gröbner basis.

In [6]:
gb_downsized = [poly.reduce(lin_polys) for poly in polys if poly.degree() > 1]
lms = [poly.lm() for poly in gb_downsized]

In [7]:
for mon in lms:
    print(mon)

x_C_3__2^2
x_C_3__3^2
x_C_3__4^2
x_C_3__5^2
x_C_3__6^2
x_C_3__7^2
x_C_3__8^2
x_C_3__9^2
x_C_3__10^2
x_C_3__11^2
x_C_3__12^2
x_C_3__13^2
x_C_3__14^2
x_C_3__15^2
x_C_3__16^2
x_C_3__17^2
x_C_3__18^2
x_C_3__19^2
x_C_3__20^2
x_C_3__21^2
x_C_3__22^2
x_C_3__23^2
x_C_3__24^2
x_C_3__25^2
x_C_3__26^2
x_C_3__27^2
x_C_3__28^2
x_C_3__29^2
x_C_3__30^2
x_C_3__31^2
x_C_3__32^2
x_C_3__33^2
x_C_3__34^2
x_C_3__35^2
x_C_3__36^2
x_C_3__37^2
x_C_3__38^2
x_C_3__39^2
x_C_3__40^2
x_C_3__41^2
x_C_3__42^2
x_C_3__43^2
x_C_3__44^2
x_C_3__45^2
x_C_3__46^2
x_C_3__47^2
x_C_3__48^2
x_C_3__49^2
x_C_3__50^2
x_C_3__51^2
x_C_3__52^2
x_C_3__53^2
x_C_3__54^2
x_C_3__55^2
x_C_3__56^2
x_C_3__57^2
x_C_3__58^2
x_C_3__59^2
x_C_3__60^2
x_C_3__61^2
x_C_3__62^2
x_C_3__63^2
x_C_3__64^2
x_C_3__65^2
x_C_3__66^2
x_C_3__67^2
x_C_3__68^2
x_C_3__69^2
x_C_3__70^2
x_C_3__71^2
x_C_3__72^2
x_C_3__73^2
x_C_3__74^2
x_C_3__75^2
x_C_3__76^2
x_C_3__77^2
x_C_3__78^2
x_C_3__79^2
x_C_3__80^2
x_C_3__81^2
x_C_3__82^2
x_C_3__83^2
x_C_3__84^2
x_C_3__85^2


In [8]:
gcds = [gcd(comb[0], comb[1]) for comb in Combinations(lms, 2)]
list(set(gcds))

[1]

Since all leading monomials are pairwise coprime we have indeed constructed a DRL Gröbner basis.

Finally, let us verify that the downsized Gröbner basis is indeed zero_dimensional by extracting the variables present in the downsized system.

In [9]:
P = gb_downsized[0].parent()
variables = P.gens()
eliminated_variables = [poly.lm() for poly in lin_polys.gens()]
variables_downsized = [var for var in variables if not var in eliminated_variables]

In [10]:
ideal([var**2 for var in variables_downsized]) == ideal(lms)

True

In [11]:
M, v = Sequence(gb_downsized).coefficients_monomials()
M = M.echelon_form()

gb_downsized_red = list((vector(M * v)))

In [12]:
sep = 100 * "-"

for poly in gb_downsized_red:
    print(poly)
    print(sep)

x_C_3__2^2 + 27919273692486402644734536198490779912*x_C_3__2*x_C_3__3 + 11884337368391687372782422775682691959*x_C_3__2*x_C_3__4 + 48933735781337299175579391034684052532*x_C_3__2*x_C_3__5 + 93590278215682651924644669128729521358*x_C_3__2*x_C_3__6 + 44319683022174854696327131885928722730*x_C_3__2*x_C_3__7 + 1157379598434881035554442043789623344*x_C_3__2*x_C_3__8 + 3668830220053225101942770400302987411*x_C_3__2*x_C_3__9 + 25577874119687566398048461481877177503*x_C_3__2*x_C_3__10 + 39633916082577856119521513392903363313*x_C_3__2*x_C_3__11 + 141136604381582061532668816183590422756*x_C_3__2*x_C_3__12 + 46340001020025006740910299473624886846*x_C_3__2*x_C_3__13 + 153852527258969749407940356966496923817*x_C_3__2*x_C_3__14 + 164634008636960080871227483705135814629*x_C_3__2*x_C_3__15 + 122669186201845469318820837001865614399*x_C_3__2*x_C_3__16 + 132515076102899492210826015157524091494*x_C_3__2*x_C_3__17 + 36882833649270037775287616195353615635*x_C_3__2*x_C_3__18 + 1441055758254442160360499386120

In [13]:
set(flatten([poly.variables() for poly in gb_downsized_red])) == set(flatten([poly.lm().variables() for poly in gb_downsized_red]))

True